In [1]:
#!/usr/bin/env python3
import sys
import copy
sys.path.append('../../')

from __future__ import print_function
from smodels.base import runtime
# Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'
# runtime.modelFile = 'mssmQNumbers.slha'

from smodels.decomposition import decomposer
from smodels.base.physicsUnits import fb, GeV, TeV
from smodels.matching.theoryPrediction import theoryPredictionsFor
from smodels.matching import theoryPrediction
from smodels.experiment.databaseObj import Database
from smodels.tools import coverage
from smodels.matching.theoryPredictionsCombiner import TheoryPredictionsCombiner
from smodels.base.smodelsLogging import setLogLevel, logger
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.model import Model
from smodels.tools.printers.masterPrinter import MPrinter
from smodels.tools import ioObjects
import os
from collections import OrderedDict
from configparser import ConfigParser, NoSectionError, NoOptionError
setLogLevel("error")

In [2]:
# Set the path to the database
database = Database('unittest')
databaseVersion = database.databaseVersion

In [3]:
parameterFile='../testPrinters_parameters.ini'
parser = ConfigParser(inline_comment_prefixes=(';',))
ret = parser.read(parameterFile)
parser.set("printer","outputType","summary")
parser.set("printer","outputFormat","current")

inputFile = '../testFiles/slha/lightEWinos.slha'
outputDir = '../'

In [4]:
parser.set("parameters", "sigmacut", "1.0")
sigmacut = parser.getfloat("parameters", "sigmacut") * fb
minmassgap = parser.getfloat("parameters", "minmassgap") * GeV

masterPrinter = MPrinter()
masterPrinter.setPrinterOptions(parser)
masterPrinter.setOutPutFiles(os.path.join(
    outputDir, os.path.basename(inputFile)))
for printer in masterPrinter.Printers.values():
    printer.output = 'stdout'


In [5]:
masterPrinter.addObj(database)

inputStatus = ioObjects.FileStatus()
if parser.getboolean("options", "checkInput"):
    inputStatus.checkFile(inputFile)

printParameters = []
if parser.has_section('parameters'):
    printParameters += list(parser.items('parameters'))
if parser.has_section('particles'):
    printParameters += list(parser.items('particles'))
if parser.has_section('options'):
    printParameters += list(parser.items('options'))

printParameters = OrderedDict(printParameters)
outputStatus = ioObjects.OutputStatus(inputStatus.status, inputFile,
                                      printParameters, databaseVersion)
masterPrinter.addObj(outputStatus)

In [6]:
model = Model(BSMparticles=BSMList, SMparticles=SMList)
promptWidth = None
stableWidth = None
if parser.has_option("particles", "promptWidth"):
    promptWidth = parser.getfloat("particles", "promptWidth")*GeV
if parser.has_option("particles", "stableWidth"):
    stableWidth = parser.getfloat("particles", "stableWidth")*GeV
model.updateParticles(inputFile=inputFile,
                      promptWidth=promptWidth, stableWidth=stableWidth)

In [7]:
smstoplist = decomposer.decompose(model, sigmacut,
                                  massCompress=parser.getboolean(
                                      "options", "doCompress"),
                                  invisibleCompress=parser.getboolean(
                                      "options", "doInvisible"),
                                  minmassgap=minmassgap)


In [8]:
masterPrinter.addObj(smstoplist)

In [9]:
allPredictions = []
combineResults = False
useBest = True
combineResults = parser.getboolean("options", "combineSRs")
allSRs = parser.getboolean("options", "reportAllSRs")
if allSRs:  # If set print out all SRs and skip combination
    useBest = False
    combineResults = False

allPredictions = theoryPredictionsFor(database,smstoplist,
                                             useBestDataset=useBest, combinedResults=combineResults)

if parser.getboolean("options", "computeStatistics"):
    for theoPred in allPredictions:
        theoPred.computeStatistics()

maxcond = parser.getfloat("parameters", "maxcond")
theoryPredictions = theoryPrediction.TheoryPredictionList(
    allPredictions, maxcond)


In [10]:
outputStatus.updateStatus(1)
masterPrinter.addObj(theoryPredictions)

In [11]:
if parser.getboolean("options", "testCoverage"):
    """ Testing coverage of model point, add results to the output file """
    if parser.has_option("options", "coverageSqrts"):
        sqrts = parser.getfloat("options", "coverageSqrts")*TeV
    else:
        sqrts = None
    uncovered = coverage.Uncovered(
        smstoplist, sigmacut=sigmacut, sqrts=sqrts)
    masterPrinter.addObj(uncovered)


In [12]:
if parser.has_option("options", "combineAnas"):
    """ Combine analyses """

    combineAnas = parser.get("options", "combineAnas").replace(" ","").split(",")
    if combineAnas:
        if combineResults is True:
            logger.warning("Combining analyses with signal region combination (combineSRs=True) might significantly reduce CPU performance.")
        combiner = TheoryPredictionsCombiner.selectResultsFrom(theoryPredictions,
                                                               combineAnas)
        # Only compute combination if at least one result was selected
        if combiner is not None:
            combiner.computeStatistics()
            masterPrinter.addObj(combiner)


In [13]:
masterCopy = copy.deepcopy(masterPrinter)
x = masterPrinter.flush()

Input status: 1
Decomposition output status: 1 #decomposition was successful
# Input File: ../testFiles/slha/lightEWinos.slha
# checkinput = True
# combineanas = ATLAS-CONF-2013-037,CMS-SUS-13-012
# combinesrs = False
# computestatistics = True
# docompress = True
# doinvisible = True
# experimentalfeatures = False
# maxcond = 0.2
# minmassgap = 5.
# model = share.models.mssm
# ncpus = 1
# promptwidth = 1e-11
# reportallsrs = False
# sigmacut = 1.0
# stablewidth = 1e-25
# testcoverage = True
# Database version: unittest300-beta
#Analysis  Sqrts  Cond_Violation  Theory_Value(fb)  Exp_limit(fb)  r  r_expected

 ATLAS-SUSY-2013-12  8.00E+00    0.0  1.403E+03  2.171E+03  6.460E-01  N/A
 Signal Region:  (UL)
 Txnames:  TChiWZoff
--------------------------------------------------------------------------------
ATLAS-CONF-2013-037  8.00E+00    0.0  4.569E-02  4.000E-01  1.142E-01  1.055E-01
 Signal Region:  SRbC3
 Txnames:  T1bbtt, T1tttt
 Likelihoods: L, L_max, L_SM =  1.891E-02,  2.067E-02, 

In [14]:
masterCopy.outputFormat = "version2"
for printer in masterCopy.Printers.values():
    printer.outputFormat = "version2"
    
x = masterCopy.flush()

Input status: 1
Decomposition output status: 1 #decomposition was successful
# Input File: ../testFiles/slha/lightEWinos.slha
# checkinput = True
# combineanas = ATLAS-CONF-2013-037,CMS-SUS-13-012
# combinesrs = False
# computestatistics = True
# docompress = True
# doinvisible = True
# experimentalfeatures = False
# maxcond = 0.2
# minmassgap = 5.
# model = share.models.mssm
# ncpus = 1
# promptwidth = 1e-11
# reportallsrs = False
# sigmacut = 1.0
# stablewidth = 1e-25
# testcoverage = True
# Database version: unittest300-beta
#Analysis  Sqrts  Cond_Violation  Theory_Value(fb)  Exp_limit(fb)  r  r_expected

 ATLAS-SUSY-2013-12  8.00E+00    0.0  1.403E+03  2.171E+03  6.460E-01  N/A
 Signal Region:  (UL)
 Txnames:  TChiWZoff
--------------------------------------------------------------------------------
ATLAS-CONF-2013-037  8.00E+00    0.0  4.569E-02  4.000E-01  1.142E-01  1.055E-01
 Signal Region:  SRbC3
 Txnames:  T1bbtt, T1tttt
 Likelihoods: L, L_max, L_SM =  1.891E-02,  2.067E-02, 